In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# data 불러오기

In [2]:
data = pd.read_csv('../data/raw_postpro.csv')
# 컬럼 삭제
df = data.drop(['Part No.2', '미입고 기간', '선박입고','완료 여부'], axis=1)

In [3]:
X = df[['Subject','Machinery', 'Assembly', 'Part No.1', '청구품목', 'key1']]
y = df['key2'].values
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [ ]:
# X_train

In [15]:
# !pip list

In [4]:
y

array([18, 18, 18, ...,  5, 20, 20])

# 데이터를 학습용과 검증용으로 나누기

In [5]:
# 데이터를 학습용과 검증용으로 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,shuffle=True)

In [6]:
# 딥러닝 모델을 만들기 위해 categorical 함수 사용
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train, num_classes=61)
y_test_categorical = to_categorical(y_test, num_classes=61)

In [7]:
# train set
features= {
   'Subject': X_train['Subject'].values,
    'Machinery':X_train['Machinery'].values ,
    'Assembly': X_train['Assembly'].values,
    # 'Part No.1' : X_train['Part No.1'].values,
    'key1' : X_train['key1'].values,
    '청구품목' : X_train['청구품목'].values
}
labels = y_train_categorical

In [8]:
# test set
predict_features = {
    'Subject': X_test['Subject'].values,
    'Machinery':X_test['Machinery'].values ,
    'Assembly': X_test['Assembly'].values,
    # 'Part No.1' : X_test['Part No.1'].values,
    'key1' : X_test['key1'].values,
    '청구품목' : X_test['청구품목'].values
}

In [11]:
# 데이터 확인
# features

In [12]:
# # df에서 각 컬럼의 단어 수 확인
# colname = ['Subject','Machinery', 'Assembly', 'Part No.1', '청구품목',]
# count = []
# for col in colname:
#     unique_subjects = set(df[col])
#     num_unique_subjects = len(unique_subjects)
#     count.append(num_unique_subjects)
#     # print(num_unique_subjects)
# print(count)

# import pandas as pd

# # 가장 긴 문자열 길이를 저장할 딕셔너리 초기화
# max_string_lengths = {}

# # 각 열 별로 길이가 가장 긴 문자열 찾기
# for col in colname:
#     max_length = df[col].str.len().max()
#     max_string_lengths[col] = max_length
# # 결과 출력
# print(max_string_lengths)

In [9]:
# 각 컬럼이 가지는 단어 갯수 확인
colname = ['Subject','Machinery', 'Assembly', 'Part No.1', '청구품목', 'key1']
unique_values = {}
for col in colname:
    unique_values[col] = X[col].unique()
    print(f"{col}: {len(unique_values[col])}")

Subject: 1716
Machinery: 174
Assembly: 1683
Part No.1: 8128
청구품목: 6129
key1: 8383


In [10]:
# colname = ['Subject','Machinery', 'Assembly', 'Part No.1', '청구품목',]
inputs = {
      'Subject': tf.keras.Input(shape=(), dtype='string'),
      'Machinery': tf.keras.Input(shape=(), dtype='string'),
      'Assembly': tf.keras.Input(shape=(), dtype='string'),
      # 'Part No.1': tf.keras.Input(shape=(), dtype='string'),
      'key1': tf.keras.Input(shape=(), dtype='string'),
      '청구품목': tf.keras.Input(shape=(), dtype='string')
    }
# size_output = tf.keras.layers.StringLookup(vocabulary=vocab)(inputs['size'])

subject_output = tf.keras.layers.StringLookup(vocabulary=unique_values['Subject'])(inputs['Subject'])
machinery_output = tf.keras.layers.StringLookup(vocabulary=unique_values['Machinery'])(inputs['Machinery'])
assembly_output = tf.keras.layers.StringLookup(vocabulary=unique_values['Assembly'])(inputs['Assembly'])
# part1_output = tf.keras.layers.StringLookup(vocabulary=unique_values['Part No.1'])(inputs['Part No.1'])
key1_output = tf.keras.layers.StringLookup(vocabulary=unique_values['key1'])(inputs['key1'])
item_output = tf.keras.layers.StringLookup(vocabulary=unique_values['청구품목'])(inputs['청구품목'])

outputs = {
      'Subject': subject_output,
      'Machinery':machinery_output,
      'Assembly': assembly_output,
      # 'Part No.1': part1_output,
          'key1': key1_output,
      '청구품목': item_output
}
preprocessing_model = tf.keras.Model(inputs, outputs)
preprocessing_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None,)]            0           []                               
                                                                                              

In [11]:
# Apply the preprocessing in tf.data.Dataset.map.
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
dataset = dataset.map(lambda x, y: (preprocessing_model(x), y),
                      num_parallel_calls=tf.data.AUTOTUNE)
# Display a preprocessed input sample.
next(dataset.take(1).as_numpy_iterator())

({'Subject': array([770], dtype=int64),
  'Machinery': array([105], dtype=int64),
  'Assembly': array([1031], dtype=int64),
  'key1': array([4280], dtype=int64),
  '청구품목': array([3390], dtype=int64)},
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       dtype=float32))

In [16]:
# print(dataset.element_spec)  # 각 요소의 구조를 확인합니다
# print(dataset.cardinality().numpy())  # 요소 개수를 확인합니다

In [17]:
# type(subject_output), subject_output.shape, type(machinery_output), machinery_output.shape,type(assembly_output), assembly_output.shape

# 모델 생성

In [12]:
inputs = {
      'Subject': tf.keras.Input(shape=(), dtype='int64'),
      'Machinery': tf.keras.Input(shape=(), dtype='int64'),
      'Assembly': tf.keras.Input(shape=(), dtype='int64'),
      # 'Part No.1': tf.keras.Input(shape=(), dtype='int64'),
      'key1': tf.keras.Input(shape=(), dtype='int64'),
      '청구품목': tf.keras.Input(shape=(), dtype='int64')
    }
# size_output = tf.keras.layers.StringLookup(vocabulary=vocab)(inputs['size'])

embedding1 = tf.keras.layers.Embedding(len(unique_values['Subject'])+1, 10)
embedding2 = tf.keras.layers.Embedding(len(unique_values['Machinery'])+1, 10)
embedding3 = tf.keras.layers.Embedding(len(unique_values['Assembly'])+1, 10)
# embedding4 = tf.keras.layers.Embedding(len(unique_values['Part No.1'])+1, 50)
embedding4 = tf.keras.layers.Embedding(len(unique_values['key1'])+1, 10)
embedding5 = tf.keras.layers.Embedding(len(unique_values['청구품목'])+1, 10)

# embedding = tf.keras.layers.Embedding(len(vocab), embedding_dims)

outputs = tf.keras.layers.Concatenate()([
        embedding1(inputs['Subject']),
        embedding2(inputs['Machinery']),
        embedding3(inputs['Assembly']),
        # embedding4(inputs['Part No.1']),
        embedding4(inputs['key1']),
        embedding5(inputs['청구품목'])
])

from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, concatenate, Flatten, Conv1D, GlobalMaxPooling1D, LSTM

# 층 추가
# outputs를 3차원으로 변환
outputs = tf.expand_dims(outputs, axis=1)  # (None, 150) -> (None, 1, 150)

# CNN Layer
cnn_output = Conv1D(filters=128, kernel_size=1, activation='relu')(outputs)  # kernel_size 수정
cnn_output = GlobalMaxPooling1D()(cnn_output)

# LSTM Layer
lstm_output = LSTM(128)(outputs)

# Dense Layer
dense_input = concatenate([cnn_output, lstm_output], axis=-1)
dense_output = Dense(64, activation='relu', name='Dense_output')(dense_input)

outputs = Dense(61, activation='softmax', name="Prediction_output")(dense_output)

# outputs = tf.keras.layers.Dense(1)(outputs)
training_model = tf.keras.Model(inputs, outputs)
training_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None,)]            0           []                               
                                                                                            

In [13]:
outputs.shape

TensorShape([None, 61])

In [14]:
# training_model.compile(
    # loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))
# training_model.compile(
#     loss= tf.keras.losses.SparseCategoricalCrossentropy(),
#     optimizer=tf.keras.optimizers.Adam(),
#     metrics=['accuracy'])

# training_model.compile(
#     loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))

training_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                       optimizer=tf.keras.optimizers.Adam(),
                       metrics=['accuracy'])

# training_model.fit(dataset)
# 모델 학습
# training_model.fit(features, y_train, epochs=10, batch_size=32)
# training_model.fit(dataset, epochs = 3, batch_size=128)
training_model.fit(dataset, epochs= 10, batch_size=32)

Epoch 1/10
14361/14361 [==============================] - 46s 3ms/step - loss: 2.2004 - accuracy: 0.4518
Epoch 2/10
 1336/14361 [=>............................] - ETA: 36s - loss: 1.1453 - accuracy: 0.7103

KeyboardInterrupt: 

In [26]:
training_model.fit(dataset, epochs=2, batch_size=32)

Epoch 1/2
14361/14361 [==============================] - 88s 6ms/step - loss: 0.1530 - accuracy: 0.9533
Epoch 2/2
14361/14361 [==============================] - 91s 6ms/step - loss: 0.0739 - accuracy: 0.9774


In [27]:
inputs = preprocessing_model.input
outputs = training_model(preprocessing_model(inputs))
inference_model = tf.keras.Model(inputs, outputs)

predict_dataset = tf.data.Dataset.from_tensor_slices(predict_features).batch(1)
y_pred = inference_model.predict(predict_dataset)

6156/6156 [==============================] - 15s 2ms/step


In [28]:
# y_pred 랑 y_test 를 원래 labeling 된 데이터로 변환
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test_categorical, axis=1)

In [ ]:
# # y_test_categorical
# print(y_train_categorical.shape)
# y_train_categorical

In [29]:
y_pred_classes[:100],y_test_classes[:100]

(array([54, 33, 24, 45, 24, 57, 41, 37,  4, 41, 57, 20, 26, 52, 26, 24, 36,
        57, 40, 30, 40, 57,  8, 36, 24,  4, 46, 37, 57, 41, 24, 24,  4,  4,
        51, 40, 20, 14, 59, 24, 26, 41, 19, 50, 24, 48, 10,  4,  7,  4,  8,
        46, 56,  7, 20, 33, 34,  2, 59, 10, 12, 48, 24, 56, 24, 24, 37,  4,
        33, 40, 22, 52, 23, 44, 45, 20, 40,  8, 56, 42, 40, 24, 24, 57, 20,
        36, 26, 24, 46, 42, 24,  8, 45, 40, 40, 21, 58, 47, 13, 14],
       dtype=int64),
 array([54, 33, 24, 45, 24, 57, 41, 10,  4, 41, 57, 20, 26, 41, 26, 24, 56,
        57, 40, 30, 40, 57,  8, 38, 24,  4, 46, 37, 57, 56, 24, 24,  4,  4,
        51, 40,  0,  6, 58, 24, 26, 42, 19, 50, 24, 48, 10,  4,  7,  4, 39,
        46, 56,  7, 20, 33, 34, 25, 58, 10, 12, 48, 24, 56, 24, 24, 37,  4,
        33, 40, 22,  8, 23, 44, 45, 20, 40,  8, 56, 42, 40, 24, 24, 45, 20,
        36, 26, 24, 46, 42, 24, 18, 45, 40, 40, 24, 58, 19, 24, 26],
       dtype=int64))

In [30]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted') # 
print("F1 Score: {:.2f}%".format(f1 * 100))

F1 Score: 80.69%


In [31]:
# y_test_classes 랑 y_pred_classes 를 원래 category 문자열로 변환
y_origin_test = le.inverse_transform(y_test)
y_origin_pred=le.inverse_transform(y_pred_classes)

In [32]:
y_origin_pred[0:100]

array(['TUBE', 'LINK', 'GASKET', 'PUMP', 'GASKET', 'VALVE', 'PIN',
       'MOTOR', 'BEARING', 'PIN', 'VALVE', 'CYLINDER', 'GEAR', 'TOOL',
       'GEAR', 'GASKET', 'MONITOR', 'VALVE', 'O-RING', 'INJECTOR',
       'O-RING', 'VALVE', 'BOLT', 'MONITOR', 'GASKET', 'BEARING', 'SCREW',
       'MOTOR', 'VALVE', 'PIN', 'GASKET', 'GASKET', 'BEARING', 'BEARING',
       'SWITCH', 'O-RING', 'CYLINDER', 'CLIP', 'WINCH', 'GASKET', 'GEAR',
       'PIN', 'COVER', 'SPRING', 'GASKET', 'SHAFT', 'BUSHING', 'BEARING',
       'BLOCK', 'BEARING', 'BOLT', 'SCREW', 'UNIT', 'BLOCK', 'CYLINDER',
       'LINK', 'LIQUID', 'ARM', 'WINCH', 'BUSHING', 'CARTRIDGE', 'SHAFT',
       'GASKET', 'UNIT', 'GASKET', 'GASKET', 'MOTOR', 'BEARING', 'LINK',
       'O-RING', 'DISC', 'TOOL', 'FILTER', 'PLATE', 'PUMP', 'CYLINDER',
       'O-RING', 'BOLT', 'UNIT', 'PIPE', 'O-RING', 'GASKET', 'GASKET',
       'VALVE', 'CYLINDER', 'MONITOR', 'GEAR', 'GASKET', 'SCREW', 'PIPE',
       'GASKET', 'BOLT', 'PUMP', 'O-RING', 'O-RING', 'DAMPER'

In [33]:
len(y_origin_test)

6156

(6156,)

In [46]:
error_category = [] # y_test 중에서 예측에 실패한 값
cnt =0
for item1, item2 in zip(y_origin_pred, y_origin_test):
    if item1 != item2:
        error_category.append(item2)
        cnt += 1
        # print(item1, item2)
        # if cnt == 50:
        #     break;
# print(cnt)

In [54]:
y_category = list(set(df['key2'].values))

In [56]:
len(y_category)

61

In [57]:
error_cnt = {}
for i in y_category:
    error_cnt[i] = 0

In [58]:
for item1, item2 in zip(y_origin_pred, y_origin_test):
    if item1 != item2:
        error_cnt[item2] +=1

In [60]:
# 각 클래스마다 예측실패한 갯수
error_cnt

{'GASKET': 216,
 'BOLT': 59,
 'TRANSFORMER': 2,
 'LINE': 3,
 'TURBINE': 3,
 'HARNESS': 5,
 'DAMPER': 8,
 'TUBE': 22,
 'LINK': 11,
 'BEARING': 43,
 'SHAFT': 30,
 'CLAMP': 12,
 'BRAKE': 7,
 'BINO': 3,
 'WIRE': 3,
 'SCREW': 24,
 'ADAPTER': 28,
 'WINCH': 8,
 'CLIP': 5,
 'UNIT': 41,
 'CARD': 0,
 'CONTROL': 12,
 'DISC': 14,
 'BATTERY': 7,
 'MOTOR': 18,
 'O-RING': 50,
 'GAUGE': 45,
 'MANIFOLD': 4,
 'VALVE': 73,
 'CONNECTING ROD': 2,
 'PIPE': 13,
 'LIQUID': 11,
 'PLATE': 20,
 'SPACER': 17,
 'MONITOR': 7,
 'COMPRESSOR': 5,
 'GENERATOR': 12,
 'WASHER': 44,
 'SPRING': 18,
 'PISTON': 10,
 'TOOL': 17,
 'CYLINDER': 40,
 'HOSE': 12,
 'BLOCK': 12,
 'PIN': 22,
 'CARTRIDGE': 11,
 'NET': 5,
 'GEAR': 43,
 'ANODES': 6,
 'BELT': 5,
 'ARM': 10,
 'COVER': 49,
 'KEY': 9,
 'FILTER': 29,
 'PUMP': 61,
 'SWITCH': 19,
 'SEAT': 0,
 'BUSHING': 12,
 'NUT': 21,
 'COOLER': 36,
 'INJECTOR': 18}

In [61]:
sorted_errors = sorted(error_cnt.items(), key=lambda x: x[1], reverse=True)

for error, count in sorted_errors:
    print(f"{error}: {count}")

GASKET: 216
VALVE: 73
PUMP: 61
BOLT: 59
O-RING: 50
COVER: 49
GAUGE: 45
WASHER: 44
BEARING: 43
GEAR: 43
UNIT: 41
CYLINDER: 40
COOLER: 36
SHAFT: 30
FILTER: 29
ADAPTER: 28
SCREW: 24
TUBE: 22
PIN: 22
NUT: 21
PLATE: 20
SWITCH: 19
MOTOR: 18
SPRING: 18
INJECTOR: 18
SPACER: 17
TOOL: 17
DISC: 14
PIPE: 13
CLAMP: 12
CONTROL: 12
GENERATOR: 12
HOSE: 12
BLOCK: 12
BUSHING: 12
LINK: 11
LIQUID: 11
CARTRIDGE: 11
PISTON: 10
ARM: 10
KEY: 9
DAMPER: 8
WINCH: 8
BRAKE: 7
BATTERY: 7
MONITOR: 7
ANODES: 6
HARNESS: 5
CLIP: 5
COMPRESSOR: 5
NET: 5
BELT: 5
MANIFOLD: 4
LINE: 3
TURBINE: 3
BINO: 3
WIRE: 3
TRANSFORMER: 2
CONNECTING ROD: 2
CARD: 0
SEAT: 0
